In [1]:
!pip install Selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 57.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [44]:
path = "C:/Users/acer/Downloads/chromedriver-win64.zip/chromedriver-win64/chromedriver"

def get_jobs(keyword, num_jobs, verbose): 
    options = webdriver.ChromeOptions()
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1120, 1000)

    
    url = 'https://www.glassdoor.com/Job/'+ keyword + '-jobs-SRCH_KO0,14.htm'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
       

        #Test for the "Sign Up" prompt and get rid of it.
        close_attempted = False
        if not close_attempted:
            try:
                WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, './/div/button[@class="CloseButton"]'))).click()
                close_attempted = True # Mark that a locking attempt has been made
            except TimeoutException:   
                pass
        

        close_cookie_hint = False
        if not close_cookie_hint:
            try:
                # Attempts to close the cookie hint
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, './/button[@id="onetrust-accept-btn-handler"]'))).click()
                close_cookie_hint = True
            except TimeoutException:
                pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements(By.XPATH, './/li[@class="JobsList_jobListItem__wjTHv"]')  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons: 

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
            
                
            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    try:
                        company_name = driver.find_element(By.XPATH, './/div/h4').text
                    except NoSuchElementException:
                        print("Company")
                        
                    try: 
                        location = driver.find_element(By.XPATH, './/div[@class="JobDetails_location__mSg5h"]').text
                    except NoSuchElementException:
                        print("Location")
                        
                    try:
                        job_title = driver.find_element(By.XPATH, './/h1[@class="heading_Heading__BqX5J heading_Level1__soLZs"]').text
                    except NoSuchElementException:
                        print("Job Title")
                    
                    collected_successfully = True
                except:
                    time.sleep(5)
                    

            try:
                salary_estimate = driver.find_element(By.XPATH, './/div[@class="SalaryEstimate_salaryRange__brHFy"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
                
            
            try:
                rating = driver.find_element(By.XPATH, './/div[@class="RatingHeadline_sectionRatingScoreLeft__di1of"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."
                

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            
            
            if not close_attempted:
                try:
                    WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, './/div/button[@class="CloseButton"]'))).click()
                    close_attempted = True # Mark that a locking attempt has been made
                except TimeoutException:   
                    pass
                
            try:
                driver.find_element(By.XPATH, './/div/button[@class="JobDetails_showMore___Le6L"]').click()
                

                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                try:
                    job_description = driver.find_element(By.XPATH, './/div[@class="JobDetails_jobDescription__uW_fK JobDetails_showHidden__C_FOA"]/child::*[1]').text
                except NoSuchElementException:
                    print("job description")
                        
                try:
                    size = driver.find_element(By.XPATH,'.//div[1]/div[@class="JobDetails_overviewItemValue__xn8EF"]').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element(By.XPATH,'.//div[2]/div[@class="JobDetails_overviewItemValue__xn8EF"]').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element(By.XPATH,'.//div[3]/div[@class="JobDetails_overviewItemValue__xn8EF"]').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element(By.XPATH,'.//div[4]/div[@class="JobDetails_overviewItemValue__xn8EF"]').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element(By.XPATH,'.//div[5]/div[@class="JobDetails_overviewItemValue__xn8EF"]').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element(By.XPATH,'.//div[6]/div[@class="JobDetails_overviewItemValue__xn8EF"]').text
                except NoSuchElementException:
                    revenue = -1

            

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1

                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element(By.XPATH, '//div[@class="JobsList_buttonWrapper__ticwb"]/button').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [45]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data scientist", 1100, False)
df

Progress: 0/1100
Progress: 1/1100
Progress: 2/1100
Progress: 3/1100
Progress: 4/1100
Progress: 5/1100
Progress: 6/1100
Progress: 7/1100
Progress: 8/1100
Progress: 9/1100
Progress: 10/1100
Progress: 11/1100
Progress: 12/1100
Progress: 13/1100
Progress: 14/1100
Progress: 15/1100
Progress: 16/1100
Progress: 17/1100
Progress: 18/1100
Progress: 19/1100
Progress: 20/1100
Progress: 21/1100
Progress: 22/1100
Progress: 23/1100
Progress: 24/1100
Progress: 25/1100
Progress: 26/1100
Progress: 27/1100
Progress: 28/1100
Progress: 29/1100
Progress: 30/1100
Progress: 31/1100
Progress: 32/1100
Progress: 33/1100
Progress: 34/1100
Progress: 35/1100
Progress: 36/1100
Progress: 37/1100
Progress: 38/1100
Progress: 39/1100
Progress: 40/1100
Progress: 41/1100
Progress: 42/1100
Progress: 43/1100
Progress: 44/1100
Progress: 45/1100
Progress: 46/1100
Progress: 47/1100
Progress: 48/1100
Progress: 49/1100
Progress: 50/1100
Progress: 51/1100
Progress: 52/1100
Progress: 53/1100
Progress: 54/1100
Progress: 55/1100
Pr

Progress: 437/1100
Progress: 438/1100
Progress: 439/1100
Progress: 440/1100
Progress: 441/1100
Progress: 442/1100
Progress: 443/1100
Progress: 444/1100
Progress: 445/1100
Progress: 446/1100
Progress: 447/1100
Progress: 448/1100
Progress: 449/1100
Progress: 450/1100
Progress: 451/1100
Progress: 452/1100
Progress: 453/1100
Progress: 454/1100
Progress: 455/1100
Progress: 456/1100
Progress: 457/1100
Progress: 458/1100
Progress: 459/1100
Progress: 460/1100
Progress: 461/1100
Progress: 462/1100
Progress: 463/1100
Progress: 464/1100
Progress: 465/1100
Progress: 466/1100
Progress: 467/1100
Progress: 468/1100
Progress: 469/1100
Progress: 470/1100
Progress: 471/1100
Progress: 472/1100
Progress: 473/1100
Progress: 474/1100
Progress: 475/1100
Progress: 476/1100
Progress: 477/1100
Progress: 478/1100
Progress: 479/1100
Progress: 480/1100
Progress: 481/1100
Progress: 482/1100
Progress: 483/1100
Progress: 484/1100
Progress: 485/1100
Progress: 486/1100
Progress: 487/1100
Progress: 488/1100
Progress: 48

Progress: 867/1100
Progress: 868/1100
Progress: 869/1100
Progress: 870/1100
Progress: 871/1100
Progress: 872/1100
Progress: 873/1100
Progress: 874/1100
Progress: 875/1100
Progress: 876/1100
Progress: 877/1100
Progress: 878/1100
Progress: 879/1100
Progress: 880/1100
Progress: 881/1100
Progress: 882/1100
Progress: 883/1100
Progress: 884/1100
Progress: 885/1100
Progress: 886/1100
Progress: 887/1100
Progress: 888/1100
Progress: 889/1100
Progress: 890/1100
Progress: 891/1100
Progress: 892/1100
Progress: 893/1100
Progress: 894/1100
Progress: 895/1100
Progress: 896/1100
Progress: 897/1100
Progress: 898/1100
Progress: 899/1100
Progress: 900/1100
Progress: 901/1100
Progress: 902/1100
Progress: 903/1100
Progress: 904/1100
Progress: 905/1100
Progress: 906/1100
Progress: 907/1100
Progress: 908/1100
Progress: 909/1100
Progress: 910/1100
Progress: 911/1100
Progress: 912/1100
Progress: 913/1100
Progress: 914/1100
Progress: 915/1100
Progress: 916/1100
Progress: 917/1100
Progress: 918/1100
Progress: 91

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist,$113K – $160K/yr (Employer est.),Company Description\n\nVisa is a world leader ...,4.1,Visa,"Washington, DC",10000+ Employees,1958,Company - Public,Information Technology Support Services,Information Technology,$10+ billion (USD)
1,Data Scientist,$104K – $145K/yr (Employer est.),"At Johnson & Johnson , we believe health is ev...",4.2,Johnson & Johnson,"Titusville, NJ",10000+ Employees,1887,Company - Public,Biotech & Pharmaceuticals,Pharmaceutical & Biotechnology,$10+ billion (USD)
2,Data Scientist,-1,Description:\nThe Data Scientist role combines...,3.4,OPERATION SMILE INC,Remote,51 to 200 Employees,1982,Nonprofit Organization,Civic & Social Services,Nonprofit & NGO,$25 to $100 million (USD)
3,Growth Data Scientist,$108K – $192K/yr (Employer est.),,4.3,Adobe,"San Jose, CA",10000+ Employees,1982,Company - Public,Computer Hardware Development,Information Technology,$10+ billion (USD)
4,TikTok Product Analysis - Data Scientist,$201K – $296K/yr (Employer est.),,3.4,TikTok,"San Jose, CA",1001 to 5000 Employees,2016,Company - Private,Internet & Web Services,Information Technology,Unknown / Non-Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...
1095,Data Scientist,$130K – $170K/yr (Employer est.),Every day we tackle new and exciting challenge...,4.3,Couchbase,"Santa Clara, CA",201 to 500 Employees,2011,Company - Public,Computer Hardware Development,Information Technology,Unknown / Non-Applicable
1096,Data Science Co-Op (NLP),$97K – $152K/yr (Glassdoor est.),Job Description\nABOUT BOSE\nYou know the mome...,3.8,Bose,"Framingham, MA",5001 to 10000 Employees,1964,Company - Private,Consumer Product Manufacturing,Manufacturing,$1 to $5 billion (USD)
1097,"Lead Data Scientist, People Analytics",$103K – $147K/yr (Glassdoor est.),"Overview:\nAs a Lead Data Scientist, you will ...",4.0,"Chick-fil-A, Inc.","Atlanta, GA",5001 to 10000 Employees,1946,Company - Private,Restaurants & Cafes,Restaurants & Food Service,$5 to $10 billion (USD)
1098,Data Scientist,$101K – $152K/yr (Glassdoor est.),Job Description\nWe are pursuing individual(s)...,4.7,Nyla Technology Solutions,"Columbia, MD",1 to 50 Employees,2013,Company - Private,State & Regional Agencies,Government & Public Administration,Unknown / Non-Applicable


In [46]:
df.to_csv('Salary Dataset.csv')
